In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

from glob import glob

%matplotlib inline

In [ ]:
data_prefix = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\summaries'
prefix = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies'
data_path = os.path.join(data_prefix, 'snowfly_data.parquet.gzip')
data = pd.read_parquet(data_path, engine='fastparquet')

In [ ]:
flyids = np.unique(data.flyid)
cols = ['filename', 'file_size', 'file_attributes',
        'region_count', 'region_id',
        'region_shape_attributes', 'region_attributes']

for fly in flyids:
    
    fly_data = data[data.flyid == fly]
    fullfile = fly_data.fullfile.iloc[0]
    folder, file = os.path.split(fullfile)
    # image_folder =  r'C:\Users\Rupp\Downloads\snapshots\snapshots' 
    image_folder = os.path.join(prefix, folder, 'snapshots')
    images = sorted(glob(os.path.join(image_folder, '*.png')))
    
    if not os.path.exists(image_folder):
        continue
    
    if not os.path.exists(os.path.join(prefix, folder, 'temp_data.csv')): 
        continue

    if not os.path.exists(os.path.join(prefix, folder, 'roi.png')): 
        continue
    
    if os.path.exists(os.path.join(image_folder, 'via_export_csv.csv')):
        continue
    
    print(fly)
    ix = 0
    im = 0
    rows = []
    
    for im in images:
    # while ix < len(fly_data)-1:
    
        # image = images[im]
        image = im
        fname = os.path.basename(image)
        size = os.path.getsize(image)    
        x = int(fly_data.x_filt.iloc[ix])
        y = int(fly_data.y_filt.iloc[ix])
        shape_attr = {'name': 'point', 'cx': x, 'cy': y}
        region_attr = {'name': 'sf'}
        row = {
            'filename': fname,
            'file_size': size,
            'file_attributes': '{}',
            'region_count': 1,
            'region_id': 1,
            'region_shape_attributes': json.dumps(shape_attr),
            'region_attributes': json.dumps(region_attr)
        }
        rows.append(row) 
        ix = ix + 60
        # im = im + 1
        
    df = pd.DataFrame(rows, columns=cols)
    out_fname = os.path.normpath(os.path.join(image_folder, 'via_export_csv.csv'))
    out_fname = out_fname.replace('\\', '/')
    df.to_csv(out_fname, index=False)

In [ ]:
## convert from via

In [ ]:
prefix = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies';

In [ ]:
flyids = np.unique(data.flyid)

for fly in flyids:

    fly_data = data[data.flyid == fly]
    fullfile = fly_data.fullfile.iloc[0]
    folder, file = os.path.split(fullfile)
    
    pos_path = os.path.join(prefix, folder, 'temp_data.csv')
    corr_path = os.path.join(prefix, folder, 'snapshots', 'via_region_data.csv')
    outpath = os.path.join(prefix, folder, 'temp_data_corrections_visible.csv')

    if not os.path.exists(pos_path) or not os.path.exists(corr_path):
        continue
        
    #if os.path.exists(outpath):
    #    continue
    
    print(outpath)
    
    pos = pd.read_csv(pos_path)
    corr = pd.read_csv(corr_path)

    ix = 0
    invisible_boundary = 60
    rows = []
    cols = ['ix', 'x', 'y', 'modified', 'visible']
    for j in range(len(corr.region_shape_attributes)):

        if (corr.region_count[j] > 1) and (corr.region_id[j] != 1):
            continue

        row = json.loads(corr.region_shape_attributes[j])
        x = row['cx']
        y = row['cy']
        modified = corr.region_count[j] > 1
        visible = (x > invisible_boundary) | (y > invisible_boundary)
        new_row = {'ix': ix, 'x': x, 'y': y, 'modified': modified, 'visible': visible}
        rows.append(new_row)
        ix = ix + 60

    df = pd.DataFrame(rows, columns=cols)

    xs = []
    ys = []
    mods = []
    vis = []

    for j in range(len(df)-1):
 
        visible = df.iloc[j].visible and df.iloc[j + 1].visible
        interpolate = df.iloc[j].modified and df.iloc[j + 1].modified
        ix = df.iloc[j].ix
        next_ix = df.iloc[j + 1].ix

        if not visible:
            vis.extend(np.zeros(60, dtype=bool))
        else:
            vis.extend(np.ones(60, dtype=bool))

        if interpolate:
            x = np.arange(ix + 1, next_ix, 1)
            xp = [ix, next_ix]
            fp_x = [df.iloc[j].x, df.iloc[j + 1].x]
            fp_y = [df.iloc[j].y, df.iloc[j + 1].y]
            xs_interp = np.interp(x, xp, fp_x)
            ys_interp = np.interp(x, xp, fp_y)
            xs.extend([fp_x[0]])
            xs.extend(xs_interp)
            ys.extend([fp_y[0]])
            ys.extend(ys_interp)
        else:
            xs.extend(list(pos.iloc[ix:next_ix].x_filt))
            ys.extend(list(pos.iloc[ix:next_ix].y_filt))
            mods.extend(~np.zeros(60, dtype=bool))

    if len(xs) < len(pos):
        remainder = len(pos) - len(xs)
        visible = df.iloc[j+1].visible
        if not visible: 
            vis.extend(np.zeros(remainder, dtype=bool))
        else:
            vis.extend(np.ones(remainder, dtype=bool))
            
        if df.iloc[j+1].modified:
            xs.extend(df.iloc[j+1].x * np.ones(remainder))
            ys.extend(df.iloc[j+1].y * np.ones(remainder))
        else:
            xs.extend(list(pos.iloc[next_ix:].x_filt))
            ys.extend(list(pos.iloc[next_ix:].y_filt))

    new_pos = {'x_new': np.array(xs).astype(int), 'y_new': np.array(ys).astype(int), 'visible': vis}
    new_df = pd.DataFrame(new_pos)
    new_df.to_csv(outpath)

G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies\1.2.21\SF0063\trial1\temp_data_corrections_visible.csv
G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies\3.18.21\SF0115\trial1\temp_data_corrections_visible.csv
